## using imbalanced-learn samplers API

In [1]:
import os
from pathlib import Path
import sys

In [2]:
project_name = 'clpsych'
project_path = Path(os.getcwd()).parent
src_path = Path(project_path, 'src')
utils_path = Path(project_path, 'utils')

if sys.platform == "win32":
    data_path = 'D:\Dataset\{0}\dataset'.format(project_name)
    embedding_path = 'D:\Dataset\{0}\embedding'.format(project_name)
    model_path = 'D:\Dataset\{0}\embedding'.format(project_name)
    
elif sys.platform == 'darwin':
    data_path = '/Volumes/Dataset/{0}/dataset'.format(project_name)
    embedding_path = '/Volumes/Dataset/{0}/embedding'.format(project_name)
    
else:
    data_path = Path(project_path, 'dataset')
    model_path = Path(project_path, 'models')
    embedding_path = Path(project_path, 'embedding')

# including the project folder and the utils folder
if str(utils_path) not in ''.join(sys.path):
    sys.path.extend([str(project_path), str(utils_path), str(src_path)])

print('project path = {0}'.format(project_path))
print('data path = {0}'.format(data_path))
print('model path = {0}'.format(model_path))
print('embedding path = {0}'.format(embedding_path))
print('sys.path = {0}'.format(sys.path))

project path = /home/guerramarj/github/clpsych
data path = /home/guerramarj/github/clpsych/dataset
model path = /home/guerramarj/github/clpsych/models
embedding path = /home/guerramarj/github/clpsych/embedding
sys.path = ['/cm/local/apps/cuda/libs/current/pynvml', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python36.zip', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/lib-dynload', '', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages/Mako-1.0.7-py3.6.egg', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages/IPython/extensions', '/home/guerramarj/.ipython', '/home/guerramarj/github/clpsych', '/home/guerramarj/github/clpsych/utils', '/home/guerramarj/github/clpsych/src']


In [3]:
# utils
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
from numpy import interp 
from collections import Counter
# evaluation metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
# optimizer
from imblearn.over_sampling import RandomOverSampler
from keras.utils import multi_gpu_model
from sklearn.preprocessing import MinMaxScaler
# ml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# deeplearning
import tensorflow as tf
from keras.layers import Reshape, Flatten, Concatenate
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Dropout
from keras import metrics
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam, Adam
from keras.regularizers import l1_l2
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
from keras.layers import Input

%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 13]

# seed for numpy and sklearn
random_state = 7
np.random.seed(random_state)

Using TensorFlow backend.


In [4]:
# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())
assert 'GPU' in str(device_lib.list_local_devices())
print('Tensorflow recognizes GPUs')

# confirm Keras sees the GPU
from keras import backend
available_gpu = backend.tensorflow_backend._get_available_gpus()
assert len(available_gpu) > 0
available_gpus = len(available_gpu)
print('number of available GPUs = {0}'.format(available_gpus))
print('list of GPUs = {0}\n'.format(available_gpu))

Tensorflow recognizes GPUs
number of available GPUs = 4
list of GPUs = ['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3']



In [5]:
def class_report(y_true, y_pred, y_score=None, average='macro'):
    if y_true.shape != y_pred.shape:
        print("Error! y_true {0} is not the same shape as y_pred {1}".format(
              y_true.shape,
              y_pred.shape)
        )
        return

    lb = LabelBinarizer()

    if len(y_true.shape) == 1:
        lb.fit(y_true)

    #Value counts of predictions
    labels, cnt = np.unique(
        y_pred,
        return_counts=True)
    n_classes = len(labels)
    pred_cnt = pd.Series(cnt, index=labels)

    metrics_summary = precision_recall_fscore_support(
            y_true=y_true,
            y_pred=y_pred,
            labels=labels)

    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average=average))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index,
        columns=labels)

    support = class_report_df.loc['support']
    total = support.sum() 
    class_report_df['avg / total'] = avg[:-1] + [total]

    class_report_df = class_report_df.T
    class_report_df['pred-cnt'] = pred_cnt
    class_report_df['pred-cnt'].iloc[-1] = total

    if not (y_score is None):
        # false positive rate
        fpr = dict()
        # true positive rate
        tpr = dict()
        roc_auc = dict()
        for label_ix, label in enumerate(labels):
            fpr[label], tpr[label], _ = roc_curve(
                (y_true == label).astype(int), 
                y_score[:, label_ix])

            roc_auc[label] = auc(fpr[label], tpr[label])

        if average == 'micro':
            if n_classes <= 2:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                    lb.transform(y_true).ravel(), 
                    y_score[:, 1].ravel())
            else:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                        lb.transform(y_true).ravel(), 
                        y_score.ravel())

            roc_auc["avg / total"] = auc(
                fpr["avg / total"], 
                tpr["avg / total"])

        elif average == 'macro':
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([
                fpr[i] for i in labels]
            ))

            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in labels:
                mean_tpr += interp(all_fpr, fpr[i], tpr[i])

            # Finally average it and compute AUC
            mean_tpr /= n_classes

            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr

            roc_auc["avg / total"] = auc(fpr["macro"], tpr["macro"])

        class_report_df['AUC'] = pd.Series(roc_auc)

    return class_report_df

In [6]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [49]:
filename = Path(data_path, 'static_features_pandas_v2.pkl')
filename = Path(data_path, 'static_features_pandas_aug_v2.2.pkl')
dataset = pd.read_pickle(filename)
filename = Path(data_path, 'extracted_features_sentiment_per_user_filtered.csv')
sentiment_statics = pd.read_csv(filename)
sentiment_statics.set_index('user_id', inplace=True)
# sentiment_statics.drop('risk_label', axis=1, inplace=True)
n_columns = ['SETLLL__{0}'.format(x) for x in sentiment_statics.columns]
sentiment_statics.columns = n_columns
print(dataset.shape)
dataset = dataset.merge(right=sentiment_statics, left_index=True, right_index=True)
print(dataset.shape)
dataset.to_pickle(Path(data_path, 'static_features_pandas_aug_v2.2.pkl'))

(496, 11574)
(496, 11614)


In [50]:
filename = 'static_features_pandas_v2.pkl'
filename = 'static_features_pandas_aug_v2.pkl'
filename = 'static_features_pandas_aug_v2.2.pkl'
dataset = pd.read_pickle(Path(data_path, filename))
print(dataset.shape)
dataset.head()

(496, 11614)


,SETAAA__Max_weekly_MH_posts_last2years,SETAAA__Max_weekly_suicide_posts_last2years,SETAAA__N_posts_alltime,SETAAA__N_posts_last2years,SETAAA__N_weeks_with_MH_posts_last2years,SETAAA__N_weeks_with_suicide_posts_last2years,SETAAA__active_days,SETAAA__avg_distance_to_2am_last2years,SETAAA__avg_post_length_last2years,SETAAA__avg_proportion_adjs_last2years,...,SETLLL__sent_4__sample_entropy,SETLLL__sent_4__skewness,SETLLL__sent_5__abs_energy,SETLLL__sent_5__absolute_sum_of_changes,"SETLLL__sent_5__linear_trend__attr_""slope""",SETLLL__sent_5__maximum,SETLLL__sent_5__mean,SETLLL__sent_5__minimum,SETLLL__sent_5__sample_entropy,SETLLL__sent_5__skewness
user_id,,,,,,,,,,,,,,,,,,,,,
16,0,1,20,20,0,2,509.961400,11.543417,505.450000,0.148389,...,1.879728,1.285680,0.031081,0.417038,0.000112,0.066699,0.035918,0.015252,2.302585,0.538883
56,0,0,115,71,0,0,905.823194,6.847304,221.338028,0.160213,...,1.905546,1.897587,0.507650,3.600498,-0.000258,0.586467,0.033911,0.002740,0.926642,8.118547
137,0,1,58,58,0,1,443.223900,13.458903,203.896552,0.184151,...,1.857388,1.515073,0.653651,4.154778,0.000533,0.536643,0.058049,0.001771,1.089579,3.630981
152,0,1,33,33,0,2,458.475648,11.730547,66.303030,0.121091,...,2.141962,1.255933,0.029821,0.575932,-0.000447,0.106684,0.022424,0.002792,1.838279,2.396103
370,1,1,75,75,5,1,582.131840,12.403656,518.253333,0.165292,...,1.929469,1.128026,0.172404,2.475222,0.000223,0.176317,0.037454,0.000622,1.883401,1.869605


In [9]:
# define 5-fold cross validation test harness
k_fold = pd.read_csv(Path(data_path, 'clpsych19_public_crossvalidation_splits.csv'), header=None,
                    names=['fold', 'train_text', 'user_id'])

# keep non conrol user ids
k_fold = k_fold[k_fold['user_id'] > 0]

<pre>
Feature Set   Description  
A             static_derieved_features  
B             post_coount_by_subreddit
C             lexicon_count  
D             sentiments_macro  
E             sentiments_micro  
F             empathy
G             readability
H             social_context
I             srl
J             ctakes
K             sentiment statics
L             sentiment statics filtered

In [43]:
data = dataset.copy()
data.shape

(496, 11574)

In [111]:
# features_interest = ['SETAAA'] 
# features_interest = ['SETBBB'] # contains NaN
# features_interest = ['SETCCC'] # contains NaN
# features_interest = ['SETDDD']
# features_interest = ['SETEEE']
# features_interest = ['SETFFF'] # contains NaN
# features_interest = ['SETGGG'] # contains NaN
# features_interest = ['SETHHH'] # contains NaN
# features_interest = ['SETIII'] # contains NaN
# features_interest = ['SETJJJ'] # contains NaN
# features_interest = ['SETKKK'] # contains NaN
# features_interest = ['SETLLL'] # contains NaN
# features_interest = ['SETAAA', 'SETDDD', 'SETEEE', 'SETFFF', 'SETGGG']
features_interest = ['SETAAA', 'SETDDD', 'SETEEE']

features = [x for x in dataset.columns if x.split('__')[0] in features_interest]
# including label
features.append('target')
data = dataset[features].copy()
data.shape

(496, 32)

In [108]:
classifier = 'nn'

ave_f1_scores = list()

print('5 fold CV starting')
for fold_ix in range(1,6):
    
    print('\nFold = {0}'.format(fold_ix))
    
    train_ix = k_fold[(k_fold['fold'] == fold_ix) & (k_fold['train_text'] == 'training')]['user_id']
    test_ix = k_fold[(k_fold['fold'] == fold_ix) & (k_fold['train_text'] == 'test')]['user_id']
    
    x_train = data[data.index.isin(train_ix)].copy()
    x_test = data[data.index.isin(test_ix)].copy()
    
    x_train = x_train.fillna(0)
    x_test = x_test.fillna(0)
    
    y_train = x_train['target']
    x_train.drop(['target'], axis=1, inplace=True)
    
    y_test = x_test['target']
    x_test.drop(['target'], axis=1, inplace=True)
    
    print(x_train.shape)
    print(sorted(Counter(y_train).items()))
    ros = RandomOverSampler(random_state=0)
    x_train, y_train = ros.fit_resample(x_train, y_train)
    print(sorted(Counter(y_train).items()))
    
    scaler = MinMaxScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    
    # one-hot encoding
    n_y_train = pd.get_dummies(y_train)
    n_y_test = pd.get_dummies(y_test)
    
    if classifier == 'nn':
        
        model = Sequential()
        model.add(Dense(30, input_dim=np.shape(x_train)[1], activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(4, activation='softmax'))
        
        adam = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
        model = multi_gpu_model(model, gpus=available_gpus)
        model.compile(optimizer=adam, loss=f1_loss,
                      metrics=[f1])
        model.fit(x_train, n_y_train, epochs=100, batch_size=100, verbose=1)
        y_pred_train = np.argmax(model.predict(x_train), axis=1) + 1
        y_pred_test = np.argmax(model.predict(x_test), axis=1) + 1
        y_score = model.predict(x_test)
        
        
        
    elif classifier == 'logit':
        model = LogisticRegression(class_weight='balanced', n_jobs=-1,
                          multi_class='auto', solver='lbfgs',
                          tol=0.00001, C=10, max_iter=1, verbose=True,
                          random_state=random_state)  
        model.fit(x_train, y_train)
        y_pred_test = model.predict(x_test)
        y_score=model.predict_proba(x_test)
        
    report_with_auc = class_report(
    y_true=y_test, 
    y_pred=y_pred_test, 
    y_score=y_score,
    average='macro')

    ave_f1_scores.append(report_with_auc['f1-score'].values[-1])

    cv_column = [fold_ix]
    cv_column.extend( [''] * (report_with_auc.index.shape[0] - 1))
    report_with_auc['Fold'] = cv_column
    report_with_auc['Risk-Factor'] = report_with_auc.index
    report_with_auc = report_with_auc.set_index(['Fold', 'Risk-Factor'])

    if fold_ix == 1:
        report_with_auc_df = report_with_auc.copy()
    else:
        report_with_auc_df = report_with_auc_df.append(report_with_auc.copy())

5 fold CV starting

Fold = 1
(395, 31)
[(1, 101), (2, 40), (3, 90), (4, 164)]
[(1, 164), (2, 164), (3, 164), (4, 164)]


/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Epoch 1/100
656/656 [==============================] - 4s 6ms/step - loss: 15.6396 - f1: 0.0000e+00
Epoch 2/100
656/656 [==============================] - 0s 57us/step - loss: 8.1283 - f1: 0.0000e+00
Epoch 3/100
656/656 [==============================] - 0s 56us/step - loss: 5.8397 - f1: 0.0027
Epoch 4/100
656/656 [==============================] - 0s 54us/step - loss: 4.6062 - f1: 0.0329
Epoch 5/100
656/656 [==============================] - 0s 59us/step - loss: 4.0564 - f1: 0.0423
Epoch 6/100
656/656 [==============================] - 0s 57us/step - loss: 3.7259 - f1: 0.0112
Epoch 7/100
656/656 [==============================] - 0s 59us/step - loss: 3.5249 - f1: 0.0245
Epoch 8/100
656/656 [==============================] - 0s 56us/step - loss: 3.5058 - f1: 0.0054
Epoch 9/100
656/656 [==============================] - 0s 55us/step - loss: 3.4538 - f1: 0.0000e+00
Epoch 10/100
656/656 [==============================] - 0s 61us/step - loss: 3.4323 - f1: 0.0000e+00
Epoch 11/100
656/656 [=

656/656 [==============================] - 0s 55us/step - loss: 3.2326 - f1: 0.0000e+00
Epoch 83/100
656/656 [==============================] - 0s 54us/step - loss: 3.2560 - f1: 0.0000e+00
Epoch 84/100
656/656 [==============================] - 0s 56us/step - loss: 3.2779 - f1: 0.0000e+00
Epoch 85/100
656/656 [==============================] - 0s 56us/step - loss: 3.2273 - f1: 0.0000e+00
Epoch 86/100
656/656 [==============================] - 0s 54us/step - loss: 3.2625 - f1: 0.0000e+00
Epoch 87/100
656/656 [==============================] - 0s 58us/step - loss: 3.2387 - f1: 0.0000e+00
Epoch 88/100
656/656 [==============================] - 0s 60us/step - loss: 3.2343 - f1: 0.0000e+00
Epoch 89/100
656/656 [==============================] - 0s 55us/step - loss: 3.2891 - f1: 0.0000e+00
Epoch 90/100
656/656 [==============================] - 0s 57us/step - loss: 3.2517 - f1: 0.0000e+00
Epoch 91/100
656/656 [==============================] - 0s 57us/step - loss: 3.2513 - f1: 0.0000e+00
Epo

/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


...

660/660 [==============================] - 0s 61us/step - loss: 3.2514 - f1: 0.0645
Epoch 85/100
660/660 [==============================] - 0s 55us/step - loss: 3.2761 - f1: 0.0546
Epoch 86/100
660/660 [==============================] - 0s 55us/step - loss: 3.2607 - f1: 0.0467
Epoch 87/100
660/660 [==============================] - 0s 54us/step - loss: 3.2709 - f1: 0.0603
Epoch 88/100
660/660 [==============================] - 0s 56us/step - loss: 3.2365 - f1: 0.0752
Epoch 89/100
660/660 [==============================] - 0s 55us/step - loss: 3.2652 - f1: 0.0633
Epoch 90/100
660/660 [==============================] - 0s 54us/step - loss: 3.2784 - f1: 0.0452
Epoch 91/100
660/660 [==============================] - 0s 58us/step - loss: 3.2226 - f1: 0.0527
Epoch 92/100
660/660 [==============================] - 0s 57us/step - loss: 3.2206 - f1: 0.0376
Epoch 93/100
660/660 [==============================] - 0s 55us/step - loss: 3.2526 - f1: 0.0315
Epoch 94/100
660/660 [=====================

/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)



Fold = 5
(398, 31)
[(1, 102), (2, 40), (3, 91), (4, 165)]
[(1, 165), (2, 165), (3, 165), (4, 165)]
Epoch 1/100
660/660 [==============================] - 4s 6ms/step - loss: 15.6499 - f1: 0.0000e+00
Epoch 2/100
660/660 [==============================] - 0s 63us/step - loss: 8.3042 - f1: 0.0000e+00
Epoch 3/100
660/660 [==============================] - 0s 57us/step - loss: 5.8666 - f1: 0.0000e+00
Epoch 4/100
660/660 [==============================] - 0s 57us/step - loss: 4.6119 - f1: 0.0022
Epoch 5/100
660/660 [==============================] - 0s 58us/step - loss: 4.0227 - f1: 0.0000e+00
Epoch 6/100
660/660 [==============================] - 0s 57us/step - loss: 3.7012 - f1: 0.0000e+00
Epoch 7/100
660/660 [==============================] - 0s 57us/step - loss: 3.4679 - f1: 0.0000e+00
Epoch 8/100
660/660 [==============================] - 0s 59us/step - loss: 3.3706 - f1: 0.0000e+00
Epoch 9/100
660/660 [==============================] - 0s 56us/step - loss: 3.3363 - f1: 0.0000e+00
Epoc

660/660 [==============================] - 0s 58us/step - loss: 3.2082 - f1: 0.0000e+00
Epoch 82/100
660/660 [==============================] - 0s 58us/step - loss: 3.1992 - f1: 0.0000e+00
Epoch 83/100
660/660 [==============================] - 0s 58us/step - loss: 3.1960 - f1: 0.0000e+00
Epoch 84/100
660/660 [==============================] - 0s 59us/step - loss: 3.1835 - f1: 0.0000e+00
Epoch 85/100
660/660 [==============================] - 0s 55us/step - loss: 3.1916 - f1: 0.0000e+00
Epoch 86/100
660/660 [==============================] - 0s 57us/step - loss: 3.1837 - f1: 0.0000e+00
Epoch 87/100
660/660 [==============================] - 0s 58us/step - loss: 3.1968 - f1: 0.0000e+00
Epoch 88/100
660/660 [==============================] - 0s 58us/step - loss: 3.1796 - f1: 0.0000e+00
Epoch 89/100
660/660 [==============================] - 0s 60us/step - loss: 3.2138 - f1: 0.0000e+00
Epoch 90/100
660/660 [==============================] - 0s 57us/step - loss: 3.2058 - f1: 0.0000e+00
Epo

/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [109]:
report_with_auc_df

precision    recall  f1-score  support  pred-cnt       AUC
Fold Risk-Factor                                                            
1    1             0.257426  1.000000  0.409449     26.0     101.0  0.500000
     avg / total   0.064356  0.250000  0.102362     26.0      26.0  0.500000
2    3             0.230000  1.000000  0.373984     23.0     100.0  0.500000
     avg / total   0.057500  0.250000  0.093496     23.0      23.0  0.500000
3    1             0.252525  1.000000  0.403226     25.0      99.0  0.587027
     avg / total   0.063131  0.250000  0.100806     25.0      25.0  0.615405
4    1             0.153846  0.080000  0.105263     25.0      13.0  0.423562
     2             0.000000  0.000000  0.000000     10.0       1.0  0.494318
     3             0.190476  0.727273  0.301887     22.0      84.0  0.644737
     avg / total   0.086081  0.201818  0.101787     57.0      57.0  0.532048
5    4             0.418367  1.000000  0.589928     41.0      98.0  0.500000
     avg / total   0.104592  0.250000  0.147482     41.0      41.0  0.500000

In [110]:
print('average f1-score (all folds) = {0}'.format(np.mean(ave_f1_scores)))

average f1-score (all folds) = 0.10918681865440867


'all'    
'SETAAA' 
'SETBBB' 
'SETCCC'
'SETDDD'
'SETEEE'
'SETFFF' 
'SETGGG' 
'SETHHH' 
'SETIII' 
'SETJJJ' 
'SETAAA', 'SETDDD', 'SETEEE', 'SETFFF', 'SETGGG'
'SETAAA', 'SETDDD', 'SETEEE', 'SETEEE'
'SETAAA', 'SETKKK' 0.20175058667560125 (droptout)
'SETAAA', 'SETLLL' 0.12430375888494909 (dropout)
'SETAAA', 'SETDDD', 'SETEEE', 'SETLLL', 'SETHHH', 'SETGGG' 0.11088291908006576 (dropout)
'SETAAA', 'SETDDD', 'SETEEE', 'SETLLL', 'SETHHH', 'SETGGG' 0.07753291443154947 (reg)
'SETAAA', 'SETDDD', 'SETEEE'  0.17349602442078949 (drop)

(nn) (dropout)
'SETAAA', 'SETDDD', 'SETEEE' 0.19014727458966665

(nn) (dropout) (1hl)
'SETAAA', 'SETDDD', 'SETEEE' 0.3255926882010475

(logit)
'SETAAA', 'SETDDD', 'SETEEE'  0.32241407748653905 
'SETAAA', 'SETDDD', 'SETEEE', 'SETLLL' 0.24496511781531444
'SETAAA', 'SETDDD', 'SETEEE', 'SETFFF', 'SETGGG' 0.11353938632270907
'SETAAA', 'SETDDD', 'SETEEE', 'SETFFF' 0.12148581335816777
'SETAAA', 'SETDDD', 'SETEEE', 'SETGGG' 0.25394562653592845